In [70]:
from nba_api.stats.endpoints import PlayByPlayV2
import pandas as pd
pd.set_option('display.max_columns', None)
from time import sleep

In [71]:
import psycopg2
from sqlalchemy import create_engine
import sqlconfig
from sqlalchemy.exc import SQLAlchemyError

In [ ]:
#SELECTING Cleaned NBA data to use with playbyplay endpoint
def read_data():
    try:
        engine = create_engine(
            f"postgresql+psycopg2://{sqlconfig.DB_USER}:{sqlconfig.DB_PASSWORD}@"
            f"{sqlconfig.DB_HOST}:{sqlconfig.DB_PORT}/{sqlconfig.DB_NAME}"
        )

        query = """WITH game AS (
            SELECT 
                ROW_NUMBER() OVER(PARTITION BY b.game_id, b.homeoraway ORDER BY b.player_id) AS rn, 
                b.game_id,
                b.homeoraway,
                b.player_id 
            FROM nba24_25.boxscores b),
            player AS (
                SELECT 
                    p.player_id,
                    p.team_id 
                FROM nba24_25.active_nba_players p
            ),
            game_team AS (
                SELECT 
                    game.game_id,
                    game.homeoraway,
                    player.team_id
                FROM game 
                LEFT JOIN player ON game.player_id = player.player_id
                WHERE game.rn = 1
            )

            SELECT 
                game_id,
                MAX(CASE WHEN homeoraway = 'Away' THEN team_id END) AS away_team_id,
                MAX(CASE WHEN homeoraway = 'Home' THEN team_id END) AS home_team_id
            FROM game_team
            GROUP BY game_id LIMIT 10;"""

        try:
            data=pd.read_sql(query, engine)
            return data
        
        except Exception as e:
            print(f"An error occurred: {e}")
        

        print("Data inserted")

    except SQLAlchemyError as e:
        print(f"Database error: {e}")

    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
def playbyplay(games):
    data_list = [] 
    for i, row in enumerate(games.itertuples(index=False), start=1):
        
            game_id = row.game_id
            # Fetch play-by-play data
            playbyplay = PlayByPlayV2(game_id)
            data = playbyplay.get_data_frames()[0]
   
            # Filter and process data
            data = data.dropna(subset=['SCORE'])  # Remove rows where SCORE is NaN
            data = data.drop_duplicates(subset=['SCORE'])  # Remove duplicate scores
     
            # Split 'SCORE' column into separate 'AWAY' and 'HOME' columns
            away_home = data['SCORE'].str.split('-', expand=True)
            away_home.columns = ['AWAY', 'HOME']
    
            # Keep only relevant columns
            data = data[['GAME_ID', 'PERIOD', 'PCTIMESTRING']]
            data = pd.concat([data, away_home], axis=1)
            row_df = pd.DataFrame([row._asdict()])  
   
            # Merge the play-by-play data with the game
            data = data.merge(row_df, how='left', left_on='GAME_ID',right_on='game_id')

            # Append data to list
            data_list.append(data)
            sleep(1)


    playbyplayscore=pd.concat(data_list)
    
    #Normalizing in to BCNF
    print("NBA_GAMES_HOME_VISITING_TEAM")
    nba_games_home_visiting_team=playbyplayscore[['GAME_ID','home_team_id','away_team_id']]
    nba_games_home_visiting_team.columns=['game_id','home_team_id','visiting_team_id']
    nba_games_home_visiting_team=nba_games_home_visiting_team.drop_duplicates()
    print(nba_games_home_visiting_team.head(5))


    print("Play By Play Score")
    playbyplayscore=playbyplayscore[['GAME_ID','PERIOD','PCTIMESTRING','HOME','AWAY']]
    playbyplayscore.columns=['game_id','current_period','time_left','htm_pts','vtm_pts']
    playbyplayscore=playbyplayscore.drop_duplicates()
    print(playbyplayscore.head(5))

    return nba_games_home_visiting_team,playbyplayscore

        

In [ ]:
def intitalInsertion(data,tablename):
    try:
        engine = create_engine(
            f"postgresql+psycopg2://{sqlconfig.DB_USER}:{sqlconfig.DB_PASSWORD}@"
            f"{sqlconfig.DB_HOST}:{sqlconfig.DB_PORT}/{sqlconfig.DB_NAME}"
        )
        
        # Insertions into the tables
        data.to_sql(tablename, engine, schema="nba24_25", if_exists="append", index=False)
      
        print("Data inserted")

    except SQLAlchemyError as e:
        print(f"Database error: {e}")

    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
games=read_data()
nba_games_home_visiting_team,playbyplayscore=playbyplay(games)
intitalInsertion(playbyplayscore,"play_by_play_score")
intitalInsertion(nba_games_home_visiting_team,"home_visiting_team")


NBA_GAMES_HOME_VISITING_TEAM
      game_id home_team_id visiting_team_id
0  0022400001   1610612738       1610612737
0  0022400002   1610612765       1610612748
0  0022400003   1610612753       1610612766
0  0022400004   1610612755       1610612752
0  0022400005   1610612752       1610612761
Play By Play Score
      game_id  current_period time_left htm_pts vtm_pts
0  0022400001               1     10:50       0      3 
1  0022400001               1     10:35       3      3 
2  0022400001               1      9:44       3      5 
3  0022400001               1      9:35       5      5 
4  0022400001               1      9:23       5      7 
Data inserted
